## Setup

In [ ]:
import ee
import eemont  # noqa: F401
import geemap
import geemap.colormaps as cm

### Authenticate Earth Engine

In [ ]:
ee.Authenticate()
ee.Initialize()

## Data and Area of interest

### Set point of interest

In [ ]:
# point = ee.Geometry.PointFromQuery('Toulouse, France', user_agent = 'user_agent')
point = ee.Geometry.Point([-108.85493802517442, 43.19884774326625]).buffer(
    10000
)  # add a buffer of 10 km around point

### Load Sentinel 2 image collection and perform preprocessing

In [ ]:
# This selects all available dates/times by default
s2 = (
    ee.ImageCollection("COPERNICUS/S2_SR")
    .filterBounds(point)
    # .filterDate('2020-01-01', '2020-01-31') # We can also include a date filter here
    .maskClouds(prob=70)
    .scaleAndOffset()
    .spectralIndices("NDVI")
)

## Data exploration

### Visualize NDVI for 1 month

In [ ]:
# Select NDVI band, filter by date range and take the median
img = s2.select("NDVI").filterDate("2020-01-01", "2020-01-31").median()

In [ ]:
# Initialize map object
m = geemap.Map()

# define visualization parameters
visNDVI = {"min": 0, "max": 1, "palette": cm.palettes.ndvi}

layer_name = "NDVI"

# add image to map, selecting NDVI band
m.addLayer(img, visNDVI, layer_name)

# Center on point
m.centerObject(point, 10)

# Add colorbar
m.add_colorbar_branca(
    colors=visNDVI["palette"],
    vmin=visNDVI["min"],
    vmax=visNDVI["max"],
    layer_name=layer_name,
)

m